## Task
1. Introduction - A brief introduction explaining what the assignment is about, including: 
     - The nature of ABC's business (real-money online gaming). 
      - Explanation of loyalty points and their importance in retaining players. 
     - The goal of the assignment (calculating loyalty points and bonus allocation). 
 
2. Problem Statement 
   - Specific tasks outlined in the assignment: 
      - Calculate player-wise loyalty points for the mentioned dates and slots. 
      - Calculate overall loyalty points and rank players. 
     - Calculate average deposit amount and number of games played. 
      - Allocate bonus to the top 50 players. 
    - Clearly mention the loyalty point formula given. 
 
 3. Data Preparation  - Importing Data: Show how you loaded the data into the Jupiter notebook, using `pandas` and 
‘numpy’   library. - Data Cleaning: Explain any steps you took to clean or preprocess the data (if applicable), such 
as handling missing values or filtering data for the required dates (e.g., 2nd, 16th, 18th, and 
26th October). 
 
 4. Loyalty Point Calculation 
    - Formulas Applied: Include Python code where the loyalty points are calculated for: 
     - Deposits: `0.01 * deposit amount` 
      - Withdrawals: `0.005 * withdrawal amount` 
      - Difference between deposits and withdrawals: `0.001 * max(#deposit - #withdrawal, 0)` 
     - Games Played: `0.2 * number of games played` - Slot-wise Calculation: Show the code and output where loyalty points are calculated per player 
for specific slots: 
       - 2nd October (S1 and S2) 
       - 16th October (S2) 
       - 18th October (S1) 
       - 26th October (S2) 
- Summarize the results, showing a table with players' names and their respective loyalty points 
for each slot. 
5. Ranking Players - Ranking Algorithm: Show the Python code where you ranked players based on their total 
loyalty points. Explain the tie-breaking mechanism using the number of games played. - Final Ranking: Present the final ranking table for the players, showing their loyalty points and 
rank. 
6. Average Calculations  - Average Deposit: Provide the Python code and results for calculating: - The overall average deposit amount. - The average deposit amount per user for the month. - Average Number of Games Played: Show the code and result for calculating the average 
number of games played per user. 
7. Bonus Allocation - Allocation Strategy: Describe your logic for distributing the Rs 50,000 bonus among the top 
50 ranked players. Include code that calculates: - The proportional distribution of the bonus based on loyalty points or another metric (like 
games played). - Final Bonus Table: Present a table showing the top 50 players with their ranks, loyalty points, 
and the bonus they receive. 
8. Discussion on Fairness - Analysis of the Current Formula: Discuss whether the current loyalty point formula is fair.  - Suggestions for Improvement: Provide recommendations to improve the fairness of the 
formula, such as: - Changing weightage between deposits, withdrawals, and games played. - Including other factors like frequency of play or player engagement. 

# Player Data Analysis for ABC Gaming Company

## Introduction
This notebook analyzes player data for ABC, an online gaming company, focusing on loyalty points and bonus distributione here


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
dates = pd.date_range(start='2024-10-01', end='2024-10-31')  # Generate a date range for October 2024
#Repeat the date range to fill 100 records
#Since there are 31 days and 100 players, we need to cycle through the dates
repeated_dates = pd.Series(np.tile(dates, 100 // len(dates) + 1)[:100])
#Simulated data for players
data = {
    'player_id': [i for i in range(1, 101)],
    'date': repeated_dates,  # Assigning repeated dates to each player
    'slot': np.random.choice(['S1', 'S2'], size=100),  # Random slot selection
    'deposit': np.random.randint(1000, 2000, 100),  # Random deposits
    'withdrawal': np.random.randint(1000, 2000, 100),  # Random withdrawals
    'games_played': np.random.randint(1000, 2000, 100),  # Random games played 
}
np.random.seed(100)
#Create a DataFrame
df = pd.DataFrame(data)
#Add deposit_count and withdrawal_count columns
df['deposit_count'] = df['deposit'].apply(lambda x: np.random.randint(1, 5))
df['withdrawal_count'] = df['withdrawal'].apply(lambda x: np.random.randint(1, 5))
#Display the DataFrame
print(df)



    player_id       date slot  deposit  withdrawal  games_played  \
0           1 2024-10-01   S2     1098        1150          1391   
1           2 2024-10-02   S1     1878        1127          1225   
2           3 2024-10-03   S1     1749        1343          1824   
3           4 2024-10-04   S1     1447        1259          1632   
4           5 2024-10-05   S1     1371        1140          1118   
..        ...        ...  ...      ...         ...           ...   
95         96 2024-10-03   S2     1076        1254          1177   
96         97 2024-10-04   S1     1734        1673          1069   
97         98 2024-10-05   S1     1343        1176          1476   
98         99 2024-10-06   S2     1276        1642          1296   
99        100 2024-10-07   S1     1823        1499          1452   

    deposit_count  withdrawal_count  
0               1                 4  
1               1                 1  
2               4                 4  
3               4              

## Part A - Calculating loyalty points

In [4]:
# Loyalty Points Formula
def calculate_loyalty_points(row):
    points_from_deposits = 0.01 * row['deposit']
    points_from_withdrawals = 0.005 * row['withdrawal']
    points_from_activity_difference = 0.001 * max(row['deposit_count'] - row['withdrawal_count'], 0)
    points_from_games = 0.2 * row['games_played']
    return points_from_deposits + points_from_withdrawals + points_from_activity_difference + points_from_games
    
# Apply the function to calculate loyalty points for each row
df['loyalty_points'] = df.apply(calculate_loyalty_points, axis=1)

df[['player_id', 'date', 'slot', 'loyalty_points']]

,player_id,date,slot,loyalty_points
0,1,2024-10-01,S2,294.930
1,2,2024-10-02,S1,269.415
2,3,2024-10-03,S1,389.005
3,4,2024-10-04,S1,347.166
4,5,2024-10-05,S1,243.010
...,...,...,...,...
95,96,2024-10-03,S2,252.431
96,97,2024-10-04,S1,239.505
97,98,2024-10-05,S1,314.511
98,99,2024-10-06,S2,280.170


## 1. Find Playerwise Loyalty points earned by Players in the following slots:-
   
 a. 2nd October Slot S1
 

In [6]:
# Filter for 2nd October, Slot S1
slot_2oct_s1 = df[(df['date'] == '2024-10-02') & (df['slot'] == 'S1')]
slot_2oct_s1[['player_id', 'loyalty_points']]

,player_id,loyalty_points
1,2,269.415
63,64,410.090


b. 16th October Slot S2 

In [8]:
# Filter for 16th October, Slot S2
slot_16oct_s2 = df[(df['date'] == '2024-10-16') & (df['slot'] == 'S2')]
slot_16oct_s2[['player_id', 'loyalty_points']]


,player_id,loyalty_points
15,16,310.965
46,47,257.671


b. 18th October Slot S1 

In [10]:
# Filter for 18th October, Slot S1
slot_18oct_s1 = df[(df['date'] == '2024-10-18') & (df['slot'] == 'S1')]
slot_18oct_s1[['player_id', 'loyalty_points']]


,player_id,loyalty_points
17,18,380.47
79,80,281.91


b. 26th October Slot S2

In [12]:
# Filter for 26th October, Slot S2
slot_26oct_s2 = df[(df['date'] == '2024-10-26') & (df['slot'] == 'S2')]
slot_26oct_s2[['player_id', 'loyalty_points']]


,player_id,loyalty_points
25,26,224.49
56,57,404.95


## 2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October.
   
 In case of tie, number of games played should be taken as the next criteria for ranking. 

In [14]:
# Calculate total loyalty points for each player across all slots
player_total_loyalty = df.groupby('player_id')['loyalty_points'].sum().reset_index()

# Rank players based on total loyalty points, breaking ties with games played
player_total_loyalty['rank'] = player_total_loyalty['loyalty_points'].rank(ascending=False, method='dense')

# Join with original dataframe to get additional player information
df = df.merge(player_total_loyalty[['player_id', 'rank']], on='player_id')

df[['player_id', 'loyalty_points', 'rank']]

,player_id,loyalty_points,rank
0,1,294.930,53.0
1,2,269.415,72.0
2,3,389.005,10.0
3,4,347.166,33.0
4,5,243.010,84.0
...,...,...,...
95,96,252.431,81.0
96,97,239.505,85.0
97,98,314.511,46.0
98,99,280.170,65.0


## 3. What is the average deposit amount?

In [16]:
# Calculate the average deposit amount
average_deposit = df['deposit'].mean()
average_deposit


1460.23

## 4. What is the average deposit amount per user in a month? 


In [18]:
# Group by player_id to calculate total deposit for each user
total_deposits_per_user = df.groupby('player_id')['deposit'].sum()

# Calculate the average deposit per user
average_deposit_per_user = total_deposits_per_user.mean()
average_deposit_per_user


1460.23

## 5. What is the average number of games played per user?

In [20]:
# Group by player_id to calculate total games played for each user
total_games_per_user = df.groupby('player_id')['games_played'].sum()

# Calculate the average number of games played per user
average_games_per_user = total_games_per_user.mean()
average_games_per_user


1443.49

# Part B - How much bonus should be allocated to leaderboard players?

After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard. The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.

Now the company needs to determine how much bonus money should be given to the players.

Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?

That’s for you to figure out.

Suggest a suitable way to divide the allocated money keeping in mind the following points: 1. Only top 50 ranked players are awarded bonus

In [22]:
# Filter for top 50 players (in this case, all players)
top_50_players = df[df['rank'] <= 50]

# Total loyalty points for all top 50 players
total_loyalty_points = top_50_players['loyalty_points'].sum()

# Calculate bonus allocation based on loyalty points
top_50_players['bonus'] = (top_50_players['loyalty_points'] / total_loyalty_points) * 50000

top_50_players[['player_id', 'loyalty_points', 'rank', 'bonus']]

,player_id,loyalty_points,rank,bonus
2,3,389.005,10.0,1077.954208
3,4,347.166,33.0,962.016042
5,6,330.801,38.0,916.667729
6,7,379.620,15.0,1051.947858
10,11,381.992,13.0,1058.520800
11,12,315.166,45.0,873.342286
13,14,359.292,29.0,995.617854
14,15,389.286,8.0,1078.732874
15,16,310.965,47.0,861.701084
17,18,380.470,14.0,1054.303254


# Part C

 ## Would you say the loyalty point formula is fair or unfair?
 
 ## Can you suggest any way to make the loyalty point formula more robust?

Analysis of the Current Formula: Discuss whether the current loyalty point formula is fair.

Suggestions for Improvement: Provide recommendations to improve the fairness of the formula, such as:

Changing weightage between deposits, withdrawals, and games played.

Including other factors like frequency of play or player engagement.